## Hello world

The code below draws a (hard-coded) triangle to the screen and contains the bare minimum to draw something with custom WebGPU shaders.

Importing webgpu.jupyter starts a pyodide kernel running alongside the notebook with access to rendering into html elements.

In [ ]:
import webgpu.jupyter as wj

The render object must be defined on the kernel side and pyodide side. For this we provide jupyter cell magic

In [ ]:
%%pyodide_and_kernel

from webgpu import RenderObject

shader_code = """

// Data structure which is the output of the vertex shader and the input of the fragment shader
struct FragmentInput {
    @builtin(position) p: vec4<f32>,
    @location(0) color: vec4<f32>,
};

// Vertex shader, returns a FragmentInput object
@vertex
fn vertex_main(
  @builtin(vertex_index) vertex_index : u32
) -> FragmentInput {

  var pos = array<vec4f, 3>(
    vec4f( 0.0,  0.5, 0., 1.),
    vec4f(-0.5, -0.5, 0., 1.),
    vec4f( 0.5, -0.5, 0., 1.)
  );

  var color = array<vec4f, 3>(
    vec4f(1., 0., 0., 1.),
    vec4f(0., 1., 0., 1.),
    vec4f(0., 0., 1., 1.)
  );
  
  return FragmentInput( pos[vertex_index], color[vertex_index] );
}

@fragment
fn fragment_main(input: FragmentInput) -> @location(0) vec4f {
  return input.color;
}
"""

class Triangle(RenderObject):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.shader_code = shader_code
        self.n_vertices = 3

    def get_shader_code(self):
        return shader_code

    def get_bindings(self):
        return []

Now you can create the object on the kernel side, fill it with data and send it with the draw command to pyodide to be rendered

In [ ]:
triangle = Triangle()
wj.Draw(triangle)